# Streamflow prediction using EE and LSTM

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/gee-community/ee-tensorflow-notebooks/blob/master/streamflow_prediction_lstm/ee_streamflow_prediction_lstm.ipynb)

This notebook provides an example workflow of how to access both observed and forcing data for hydrologic modeling and train a Long-Short-Term Memory (LSTM) model for modeling historical streamflow and predicting future streamflow. We use Earth Engine to access meteorological data as inputs into the model.

This example is insprired by the following paper: [Rainfall–runoff modelling using Long Short-Term Memory (LSTM) networks](https://www.hydrol-earth-syst-sci.net/22/6005/2018/)

## Setting up the environment

### Importing packages

Here we use Pandas to handle our time series data that we are going to use to train our model and some HTTP request/IO pacakges to get data from the USGS.

In [0]:
%pylab inline

import pandas as pd
import requests
from io import StringIO

In [0]:
# Import, authenticate and initialize the Earth Engine library.
import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [0]:
# Tensorflow setup.
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K

# used to help transform data
from sklearn import preprocessing

print(f"Using TensorFlow version {tf.__version__}")

In [0]:
import folium

## Accessing data

For this example we will need two datasets:
1. Observed streamflow - this can sometimes be difficult to access but the USGS provides an website (https://waterdata.usgs.gov/nwis) to where you can search and even programmitically access gauge data
2. Meteorological forcing data - this is meteorological data for a watershed that will be used to model streamflow


### Observed streamflow

To access the observed streamflow, we will send a request to get a CSV table. We will need to specify a few parameters though including: time period and site number.

Time period will be used to get the observed streamflow and forcing data. Whereas the site number is gauge specific and you will need to search and replace this at the `siteNumber` variable. For this example we will use a watershed in the Appalachian Mountain Range, the [Pigeon River Watershed](https://waterdata.usgs.gov/nwis/inventory?agency_code=USGS&site_no=03461500). *FUN FACT*: This area is also a great place to go camping!

You can search for a site of interest and accessing the data/gauge information using the [USGS National Water Information System Mapper](https://maps.waterdata.usgs.gov/mapper/index.html)

In [0]:
# change to your prefered study period
# must be within the time range of available observed data
START_TIME = '1997-01-01'
END_TIME = '2011-12-31'

In [0]:
# change to the site number of interest
siteNumber = '03461500'

usgsWaterData = 'https://waterdata.usgs.gov/nwis/dv'
params = dict(
    site_no=siteNumber,
    begin_date=START_TIME, # start time from variable
    end_date=END_TIME, # end time from variable
    format='rdb', # default parameter, do not change
)

# send request
x = requests.get(usgsWaterData,params=params)

if x.status_code == 200:
    print('Data request succeeded! 🎉')
else:
    print("Uh-oh...something went wrong.")

The returned data comes formatted as text with some header information so first we will have to parse the header from the actual data

In [0]:
# split the response text into the header information and table data
header,table = x.text.split('# \n')

In [0]:
print(header)

After we have parsed the data from the response, we can load it into a Pandas DataFrame for processing.

In [0]:
colNames = ['agency','site','datetime','discharge','quality']
df = pd.read_csv(StringIO(table),sep='\t')
df = df.iloc[1:]
df.columns = colNames

print(f'Table dimensions: {df.shape}')
df.head()

Metric units are used all over the world (except the US...) and in science so we will convert the units from cubic feet per second to cubic meters per second.
Also we will index the DataFrame by the `datetime` information so we can do some time series processing on it.

In [0]:
df.index = pd.to_datetime(df['datetime'])
df = df['discharge'].astype(np.float32) * 0.0283168 # convert to m^3/s

## Forcing data

Our next step is to access forcing data, this is typically meteorological data. Earth Engine provides great functionality to access vast amounts of met data and format it in a way that can directly be used for this application. To do this we will need the geometry of the watershed and the image collection of met data.

Here we specify the location of the gauge station and select from the HydroSheds FeatureCollection which one intersects with the gauge.

*CAUTION*: Typically for lumped hydrologic modeling you will want the gauge be as close to the watershed outlet as possible. If you change the watershed of interest be sure to carefully select which level of HydroSheds makes sense.

In [0]:
# specify where the gauge is located so we can filter the basin by location
gaugeLat,gaugeLon = 35.96055556, -83.17444444
gaugeGeom = ee.Geometry.Point([gaugeLon,gaugeLat])

In [0]:
hydrosheds = ee.FeatureCollection("WWF/HydroSHEDS/v1/Basins/hybas_8")

pigeonRiverBasin = hydrosheds.filterBounds(gaugeGeom)

Just so we can see where we are at in the world, let's map the watershed.

In [0]:
basinOutline = ee.Image().byte()\
    .paint(
        featureCollection= pigeonRiverBasin,
        color= 1,
        width= 3
    ).getMapId()

map = folium.Map(location=[gaugeLat,gaugeLon],zoom_start=9,height=750)
folium.TileLayer(
    tiles=basinOutline['tile_fetcher'].url_format,
    attr='Google Earth Engine',
    overlay=True,
    name='Pigeon River Basin',
  ).add_to(map)

folium.Marker([gaugeLat,gaugeLon]).add_to(map)

map.add_child(folium.LayerControl())
map

Now we need to get the meterological image collection, for this we will use the [daily ERA5 dataset](https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY) and select the precipitation, minimum/maximum temperature, and wind variables.

In [0]:
# specify band names we want
metBands = ['total_precipitation','minimum_2m_air_temperature','maximum_2m_air_temperature','u_component_of_wind_10m','v_component_of_wind_10m']

# filter the collection by date and select the bands on interest
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY')\
    .filterDate(START_TIME,ee.Date(END_TIME).advance(1,'day'))\
    .select(metBands)

All that is left to do is get the data. `imageCollection.getRegion()` is a handy function that allows us to request all of the data over a geometry from an imageCollection. Then we use `.getInfo()` to get the data on the client side.

*CAUTION*: When using `.getInfo()` you can easily run into memory limit errors. One way to avoid this is to limit your time period you are looking at or request data at a larger scale.

In [0]:
scale = 30000 # request scale in meters
lumpedForcings = era5.getRegion(pigeonRiverBasin,scale).getInfo()

The resulting data comes in a specific format that will need to be organized. Lukily, pandas handles this nicely so we will create a DataFrame of the pixel information.

In [0]:
forcingDf = pd.DataFrame(lumpedForcings[1:])
forcingDf.columns = lumpedForcings[0]
forcingDf.head()

If there are more than one pixels per time period then we will need to aggregate to create a mean value for each variable per time period. We do this using `.groupby()` by the imageId and specifying statistics to aggregate by.

Also, we will do some manipulation to the dataframe to make it a time series and name the columns something a little more intuitive.

In [0]:
forcingDf = forcingDf.groupby(by='id')\
    .agg({'total_precipitation':'mean',
          'minimum_2m_air_temperature':'mean',
          'maximum_2m_air_temperature':'mean',
          'u_component_of_wind_10m':'mean',
          'v_component_of_wind_10m':'mean',
          'time':'mean'
         }) 

forcingDf.index = pd.to_datetime(forcingDf['time']*1e6)
forcingDf.index.name = 'datetime'
forcingDf.drop(['time'],axis=1,inplace=True)

newCols = ['precip','tmin','tmax','uwind','vwind']
forcingDf.columns = newCols

In [0]:
forcingDf.head()

Great, nicely formatted! Now we can concatenate both the observed streamflow and forcing data into a single dataframe. This will be handy when we have to format the data for the LSTM model to keep track of everything together.

In [0]:
modelDf = pd.concat([df,forcingDf],axis=1)
modelDf.dropna(inplace=True)

modelDf.head()

## Building and training the model

Now we have the data we can start working on our LSTM model. LSTM's take data in a spefic format that support the time component of the model: [samples,features,time].

Now we have samples and features but we will need to create lagged arrays along a third dimension to provide the time component. Here we define a custom function to do this for use.

In [0]:
def dataPrep(df,featureNames,labelNames,timeLag=10,predLead=0,scalingFunc=None,pctTrain=0.77):
    """
    Function to pred dataframe for input into a RNN

    Args:
        df: dataframe with features and label
        featureNames: list of column names that will be the input features
        labelNames: list of column names that will be the output labels
    Kwargs:
        timeLag: time to lag datasets default = 10 periods
        predLead: time period as forecast ouputs
        scalingFunc: sklearn.preprocessing function to preprocess features
        pctTrain: percent of data to be used for training, used to prevent scaling on all features
    Returns:
        X_train: array of training features
        X_test: array of testing features
        y_train: array of training labels
        y_test: array of testing labels
    """

    # get features
    x = df[featureNames].values

    # get the labels
    if predLead > 0:
        y = df[labelNames][timeLag:-predLead].values
    else:
         y = df[labelNames][timeLag:].values

    nTrain = int(pctTrain*x.shape[0])

    # if a scaling function is provided then scale the data
    if scalingFunc is not None:
        scaler = scalingFunc.fit(x[:nTrain])
        x_scaled = scaler.transform(x)
    else:
        x_scaled = x

    xshape = [y.shape[0]]+[x.shape[1]]+[timeLag]
    yshape = [y.shape[0],predLead] if predLead > 0 else [y.shape[0],1]
    outx = np.zeros(xshape)
    outy = np.zeros(yshape)
    for i in range(y.shape[0]-predLead):
        v = timeLag+i if i >0 else timeLag
        u = predLead+i if i>0 else predLead
        u = u if predLead > 0 else i+1
        outx[i,:,:] = x_scaled[i:v,:].T
        outy[i,:] = y[i:u].T

    nTrain = int(pctTrain*outx.shape[0])

    xtrain,xtest = outx[:nTrain,:,:],outx[nTrain:,:,:]
    ytrain,ytest = outy[:nTrain,:],outy[nTrain:,:]

    return xtrain,xtest,ytrain,ytest



Now we have a function to prepare our data. Let's use it.

Here we define some of the parameters that will be passed in our `dataPred()` function. We use 365 days as input into the model, all the met data, and 85% of the data for training. 

In [0]:
timeDim = 365
featureColumns = ["precip","tmin","tmax","uwind","vwind"]
labelColumns = ['discharge']
pctTrain = 0.85

scaler = preprocessing.RobustScaler()

X_train,X_test,y_train,y_test = dataPrep(modelDf,featureColumns,labelColumns,timeLag=timeDim,scalingFunc=scaler,pctTrain=pctTrain)

Let's check to make sure the data is properly formatted.

In [0]:
print(X_train.shape,y_train.shape)

We can see that the `X_train` array does in fact have 3 dimensions with the samples, features, and time information and our `y_train` array matches the samples length.

Let's next define a function to build our model. This model will be a simple LSTM model with two LSTM layers with ReLU activations and an output layer. The function will compile the model and return it ready to use. 

In [0]:
def buildModel(inshape,outshape,nodes=32,optimizer='adam',loss='mse'):
    """
    Function to build out LSTM model

    Args:
        inshape: list or Tensor defining input shape for model must have [features,time]
        outshape: int defining output shape for model must have [time]
    Kwargs:
        nodes: number of nodes to use in LSTM layers, default = 32
        optimizer: string or optimizer to use for model, default = adam optimizer
        loss: string of loss function to use for model, default = mean squared error
    Returns: 
        model: compiled Keras model ready for training
    """
    inputs = layers.Input(inshape,name="input_layer")
    x = layers.LSTM(nodes,return_sequences=True,name="lstm_layer_1",activation="relu")(inputs)
    x = layers.LSTM(nodes,return_sequences=False,name="lstm_layer_2",activation="relu")(x)
    outputs = layers.Dense(outshape,activation="linear",name="output_layer")(x)

    model = models.Model(inputs=[inputs],outputs=[outputs],name="hydroNet")

    model.compile(optimizer=optimizer,
                 loss=loss,
                 metrics=["mape","mae","mse"])

    return model

In hydrologic modeling it is commom to use a metric called the Nash-Sutcliffe model efficiency coefficient (NSE). So, here we define a custom loss function to calculate NSE and since the NSE values range from -$\infty$ to 1 where 1 is a perfect model. However, for loss functions lower values are better so we will invert the values so that -1 is best.

In [0]:
def nse_loss(y_true, y_pred):
    """
    Custom metric function to calculate the Nash-Sutcliffe model efficientcy coefficient
    From: https://en.wikipedia.org/wiki/Nash%E2%80%93Sutcliffe_model_efficiency_coefficient
    Commonly used in hydrology to evaluate a model performance (NSE > 0.7 is good)
    
    Args:
        y_true: Tensor with true values from observations/labels
        y_pred: Tensor of predicted values from model
    Returns: 
       tf.Tensor of the inverted NSE value
    """
    numer = K.sum(K.pow(y_true-y_pred,2))
    denom = K.sum(K.pow(y_true-K.mean(y_true),2)) + K.epsilon()
    nse = (1 - (numer/denom))
    return -1*nse


Now we can build our model. We define the input and output shape parameters and provide the custom `nse_loss` function for the loss.

In [0]:
inshape = len(featureColumns),timeDim
outshape = 1

model = buildModel(inshape=inshape,outshape=outshape,loss=nse_loss)
model.summary()

In [0]:
EPOCHS = 15
STEPS_PER_EPOCH = 500
BATCH_SIZE = X_train.shape[0]//STEPS_PER_EPOCH
VAL_SPLIT = 0.1

In [0]:
training = model.fit(x=X_train,y=y_train,
                     epochs=EPOCHS,
                     batch_size=BATCH_SIZE,
                     validation_split=VAL_SPLIT)

We have a trained model so now let's look at what happened during training. Here we plot the history and inspect the loss and some metric values for each epoch.

In [0]:
fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(10,5.5))

ax[0].plot(training.history['loss'],color='#1f77b4',label='Training Loss')
ax[0].plot(training.history['val_loss'],linestyle=':',marker='o',markersize=3,color='#1f77b4',label='Validation Loss')
ax[0].set_ylabel('Loss')
ax[0].legend()

ax[1].plot(training.history['mae'],color='#ff7f0e',label='MAE')
ax[1].plot(training.history['val_mae'],linestyle=':',marker='o',markersize=3,color='#ff7f0e',label='Validation MAE')
ax[1].set_ylabel('Mean Absolute Error')
ax[1].set_xlabel('Epoch')
ax[1].legend(loc="lower right")

ax[1].set_xticks(range(1,len(training.epoch)+1,5))
ax[1].set_xticklabels(range(1,len(training.epoch)+1,5))
ax[1].set_xlabel('Epoch')

plt.legend()

plt.show()

In addition to inpecting the training histoy, we can also predict the streamflow from the test samples and evaluate.

In [0]:
# apply the prediction
y_pred = model.predict(X_test)

# drop the extra dimension for the prediction and test arrays
# this is done for plotting
y_pred = np.squeeze(y_pred) 
y_test = np.squeeze(y_test) 

In [0]:
# calculate some metrics to see how well are predictions are doing
rmse = np.mean(np.sqrt(np.power((y_test-y_pred),2)))
nse = 1-(np.sum(np.power((y_test-y_pred),2))/np.sum(np.power((y_test-y_test.mean()),2)))

Now we can plot our results to see inspect how our model performs on data it has not seen before.

In [0]:
print(f'NSE: {nse:.4f}  RMSE: {rmse:.4f}')

eval_dates = modelDf.iloc[X_train.shape[0]+timeDim:].index

fig = plt.figure(figsize=(25,8))
gs = fig.add_gridspec(1, 3)

ax1 = fig.add_subplot(gs[0, :2])
ax1.plot(eval_dates, y_pred,label='Predicted')
ax1.plot(eval_dates, y_test,label='Observed')
ax1.set_xlabel('Date')
ax1.set_ylabel('Discharge')
ax1.legend(fontsize=12)

ax2 = fig.add_subplot(gs[0, 2:])
ax2.plot(y_pred,y_test,'o',alpha=0.4)
ax2.plot([0,y_test.max()],[0,y_test.max()],'k--',alpha=0.75)
ax2.set_xlabel('Predicted')
ax2.set_ylabel('Observed')

plt.show()

Alright, not too bad for a hindcast. There are plenty of ways to customized the model to increase accuracy. This serves largely as an small example to get started and there are plenty of ways to customize for your own application.

## Forecasting streamflow

We have seen an example of how we can use an LSTM for generating already observed streamflow but what if we want to forecast? We can do that too, all it takes is a little adjustment for the input data.

To perfrom a forecast, we will specify a forecastLead variable which will be number of days to forecast and adjust our `timeLag` to 60 days (most forecasts are not influenced by more than two months worth of data). Then we get our training and testing data with the new dimensions.

In [0]:
forecastLead = 15
timeDim = 60

X_train,X_test,y_train,y_test = dataPrep(modelDf,featureColumns,labelColumns,timeLag=timeDim,predLead=forecastLead,scalingFunc=scaler,pctTrain=pctTrain)

In [0]:
# check the shape matches what we would expect
print(X_train.shape,y_train.shape)

We can now build the forecast model with and provide the dimensions of the input data. We are going to use the same model build from earlier but provide a few more nodes in each LSTM layer.

In [0]:
inshape = len(featureColumns),timeDim
outshape = forecastLead

model = buildModel(inshape=inshape,outshape=outshape,nodes=48,loss=nse_loss)
model.summary()

Now we fit the model.

In [0]:
model.fit(x=X_train,y=y_train,
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          validation_split=VAL_SPLIT)

In [0]:
# run the predictions
forecast = model.predict(X_test)
forecast.shape # print the shape

Alright, we have the model and prediction outputs from the model. Now let's see how it worked...but looking at time series forecasts is not straightforward!

We will use some IPython widget action to help us out. Here we define a function to plot our forecasts. We use a date picker to help visualize the forecast for that day then plot the historical streamflow with the predicted and true streamflow based on the forecast date.

In [0]:
from ipywidgets import *

eval_dates = modelDf.iloc[X_train.shape[0]+timeDim:].index
initDate = datetime.datetime(2010,5,1)

@widgets.interact(date=DatePicker(description='Pick a Date:',value=initDate))
def plotUpdate(date):
    fIdx = int(np.where(eval_dates==str(date))[0])
    fig,ax = plt.subplots(1,1,figsize=(15,8))
    historicLine, = ax.plot(eval_dates[fIdx-50:fIdx],y_test[fIdx-50:fIdx,0],label="historic",alpha=0.5)
    predictedLine, = ax.plot(eval_dates[fIdx:fIdx+forecastLead],forecast[fIdx,:],color="C1", marker='o',label="predicted")
    truthLine, = ax.plot(eval_dates[fIdx:fIdx+forecastLead],y_test[fIdx:fIdx+forecastLead,0],color="C0", marker='o',label="truth")
    ax.set_title(f'Forecast for {date}',fontsize=12)
    ax.set_ylabel('Discharge [m^3/s]')
    plt.legend(fontsize=10)
    plt.show()


Based on these results, I would not be putting any money on the predictions.... Again, this serves more as an example rather than a production ready model.

## Conclusion

In this example we looked at how we can use Earth Engine and TensorFlow to predict streamflow. We accessed observed streamflow from USGS, used Earth Engine to get meteorological data for our watershed, and built LSTM models for hindcast and forecasts of streamflow. While these examples do not produce the best results, they serve as a starting point to where you can create streamflow predictions using deep learning. If you would want to run this in production then you could write functions to call the data from EE and run the predictions in real-time!